In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x6aff240


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
# plot_path = "plots/angles/mg5_1M"
plot_path = "/dev/null"
oo_plot_name = "OO_wz.pdf"

In [5]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [6]:
ROOT.gInterpreter.Declare("#include <WWTools.h>")
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools.h\"")

True

In [7]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882

ROOT.OOTools.init(250, mW, mZ)

In [8]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [9]:
%%cpp
using namespace ROOT::VecOps;

In [10]:
df = ROOT.RDataFrame("events", "data/gen/whizard_ww_semi/ww.edm4hep.root")
df.Describe()

Dataframe from TChain events in file data/gen/whizard_ww_semi/ww.edm4hep.root

Property                Value
--------                -----
Columns in total           50
Columns from defines        0
Event loops run             0
Processing slots            1

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                   ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeader.weight                      ROOT::VecOps::RVec<Double_t>                    Dataset
EventHeader.weights_begin               ROOT::VecOps::RVec<UInt_t>    

In [11]:
# these files are very simple, they have the Ws explicitly and there is only one stable e/p
df = df.Define("e_MC_idx", "ArgMax(MCParticles.PDG == 11 && MCParticles.generatorStatus == 1)")
df = df.Define("enu_MC_idx", "ArgMax(MCParticles.PDG == -12 && MCParticles.generatorStatus == 1)")
df = df.Define("p_MC_idx", "ArgMax(MCParticles.PDG == -1 && MCParticles.generatorStatus == 1)")
df = df.Define("pnu_MC_idx", "ArgMax(MCParticles.PDG == 2 && MCParticles.generatorStatus == 1)")
df = df.Define("Wp_MC_idx", "ArgMax(MCParticles.PDG == 24)")
df = df.Define("Wm_MC_idx", "ArgMax(MCParticles.PDG == -24)")

df = df.Define("e_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[e_MC_idx], MCParticles.momentum.y[e_MC_idx], MCParticles.momentum.z[e_MC_idx], MCParticles.mass[e_MC_idx])")
df = df.Define("enu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[enu_MC_idx], MCParticles.momentum.y[enu_MC_idx], MCParticles.momentum.z[enu_MC_idx], MCParticles.mass[enu_MC_idx])")
df = df.Define("p_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[p_MC_idx], MCParticles.momentum.y[p_MC_idx], MCParticles.momentum.z[p_MC_idx], MCParticles.mass[p_MC_idx])")
df = df.Define("pnu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[pnu_MC_idx], MCParticles.momentum.y[pnu_MC_idx], MCParticles.momentum.z[pnu_MC_idx], MCParticles.mass[pnu_MC_idx])")
df = df.Define("Wp_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wp_MC_idx], MCParticles.momentum.y[Wp_MC_idx], MCParticles.momentum.z[Wp_MC_idx], MCParticles.mass[Wp_MC_idx])")
df = df.Define("Wm_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wm_MC_idx], MCParticles.momentum.y[Wm_MC_idx], MCParticles.momentum.z[Wm_MC_idx], MCParticles.mass[Wm_MC_idx])")

In [12]:

df = df.Define("l_MC_lvec_e", "e_MC_lvec.energy()")
df = df.Define("l_MC_lvec_theta", "e_MC_lvec.theta()")
df = df.Define("l_MC_lvec_cosTheta", "cos(l_MC_lvec_theta)")

df = df.Define("l_MC_lvec_pt", "e_MC_lvec.pt()")
df = df.Define("l_MC_lvec_pz", "e_MC_lvec.pz()")

df = df.Define("e_MC_lvec_e", "e_MC_lvec.energy()")
df = df.Define("enu_MC_lvec_e", "enu_MC_lvec.energy()")
df = df.Define("p_MC_lvec_e", "p_MC_lvec.energy()")
df = df.Define("pnu_MC_lvec_e", "pnu_MC_lvec.energy()")

df = df.Define("e_MC_lvec_px", "e_MC_lvec.px()")
df = df.Define("e_MC_lvec_py", "e_MC_lvec.py()")
df = df.Define("e_MC_lvec_pz", "e_MC_lvec.pz()")

df = df.Define("p_MC_lvec_px", "p_MC_lvec.px()")
df = df.Define("p_MC_lvec_py", "p_MC_lvec.py()")
df = df.Define("p_MC_lvec_pz", "p_MC_lvec.pz()")

df = df.Define("Wm_MC_lvec_px", "Wm_MC_lvec.px()")
df = df.Define("Wm_MC_lvec_py", "Wm_MC_lvec.py()")
df = df.Define("Wm_MC_lvec_pz", "Wm_MC_lvec.pz()")

df = df.Define("Wp_MC_lvec_px", "Wp_MC_lvec.px()")
df = df.Define("Wp_MC_lvec_py", "Wp_MC_lvec.py()")
df = df.Define("Wp_MC_lvec_pz", "Wp_MC_lvec.pz()")

### start of angle calculations
# the simplest is the production angle theta_W (where W stands not for Weinberg in this case but for W-boson)
# following M. Bilenky et al. in Nucl.Phys.B 409 (1993) 22-68 (doi: https://doi.org/10.1016/0550-3213(93)90445-U)
# this is the angle between the W^{-} and the beam e^{-} which in our case is just the Z-axis
# (TODO: check what happens when ddsim adds a crossing angle :/)

df = df.Define("W_minus_lvec", "Wm_MC_lvec")
df = df.Define("W_plus_lvec", "Wp_MC_lvec")
df = df.Define("e_minus_vec", "ROOT::Math::XYZVector(0, 0, 1)")
df = df.Define("e_plus_vec", "ROOT::Math::XYZVector(0, 0, -1)")

df = df.Define("CosTheta_W_minus", "ROOT::Math::VectorUtil::CosTheta(W_minus_lvec, e_minus_vec)")
# FIXME: e_minus_vec?
df = df.Define("CosTheta_W_plus", "ROOT::Math::VectorUtil::CosTheta(W_plus_lvec, e_minus_vec)")


In [13]:
df = df.Define("enu_m", "(e_MC_lvec + enu_MC_lvec).M()")
df = df.Define("pnu_m", "(p_MC_lvec + pnu_MC_lvec).M()")

In [14]:
# df = df.Define("e_transformed_lvec", "WWTools::starVectorOO(W_minus_lvec, e_MC_lvec)")
# df = df.Define("p_transformed_lvec", "WWTools::starVectorOO(W_plus_lvec, p_MC_lvec)")
# df = df.Define("e_transformed_lvec", "WWTools::starVector(W_minus_lvec, e_MC_lvec)")
# df = df.Define("e_transformed_lvec", "WWTools::starVector(W_minus_lvec, e_MC_lvec, {0, 0, 1})")
# df = df.Define("p_transformed_lvec", "WWTools::starVector(W_plus_lvec, p_MC_lvec)")
df = df.Define("e_transformed_lvec", "WWTools::starVectorHagiwara(W_minus_lvec, e_MC_lvec, {0, 0, 1}, W_minus_lvec)")
df = df.Define("p_transformed_lvec", "WWTools::starVectorHagiwara(W_plus_lvec, p_MC_lvec, {0, 0, 1}, W_minus_lvec)")

In [15]:

df = df.Define("e_theta_prime", "e_transformed_lvec.Theta()")
df = df.Define("e_phi_prime", "e_transformed_lvec.Phi()")
# df = df.Define("e_phi_prime", "e_transformed_lvec.Phi() + ROOT::Math::Pi()")
df = df.Define("p_theta_prime", "p_transformed_lvec.Theta()")
df = df.Define("p_phi_prime", "p_transformed_lvec.Phi()")
# df = df.Define("p_phi_prime", "p_transformed_lvec.Phi() + ROOT::Math::Pi()")
# df = df.Define("l_theta_prime", "auto theta = l_transformed_lvec.Theta(); return l_MC_charge == 1. ? ROOT::Math::Pi() - theta : theta;")
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); return l_MC_charge == 1. ? phi - ROOT::Math::Pi() : phi;")

# OPAL shift W plus by 180 deg, but this gives it the wrong shape?!
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); auto phi_s = l_MC_charge == 1. ? phi + ROOT::Math::Pi() : phi; return phi_s > ROOT::Math::Pi() ? phi_s - 2 * ROOT::Math::Pi() : phi_s;")
# df = df.Define("p_phi_prime", "auto phi = p_transformed_lvec.Phi() + ROOT::Math::Pi(); return phi > ROOT::Math::Pi() ? phi - 2 * ROOT::Math::Pi() : phi;")

df = df.Define("e_cosTheta_prime", "cos(e_theta_prime)")
df = df.Define("p_cosTheta_prime", "cos(p_theta_prime)")
# df = df.Define("p_cosTheta_prime", "-cos(p_theta_prime)")

In [16]:
df = df.Define("aw", "OOTools::aw(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("awph", "OOTools::awph(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("abph", "OOTools::abph(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

df = df.Define("dg1z", "OOTools::dg1z(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("dkg", "OOTools::dkg(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("lz", "OOTools::lz(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

df = df.Define("fg1", "OOTools::fg1(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fg2", "OOTools::fg2(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fg3", "OOTools::fg3(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fg4", "OOTools::fg4(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fg5", "OOTools::fg5(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fg6", "OOTools::fg6(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fg7", "OOTools::fg7(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

df = df.Define("fz1", "OOTools::fz1(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fz2", "OOTools::fz2(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fz3", "OOTools::fz3(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fz4", "OOTools::fz4(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fz5", "OOTools::fz5(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fz6", "OOTools::fz6(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
df = df.Define("fz7", "OOTools::fz7(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

In [17]:
# FIXME: for debugging
df = df.Define("weight", "1")
# df = df.Filter("l_MC_lvec_pt > 2", "pt cut")
# df = df.Filter("abs(l_MC_lvec_cosTheta) < 0.975", "cosTheta cut") # ~6.5 deg
# df = df.Filter(f"abs(enu_m - {mw}) < 2. && abs(pnu_m - {mw}) < 2.")

In [18]:
oo_bins = 250
oo_start = -1.
oo_end = 1.5

h_aw = df.Histo1D(("", ";O_{#alphaW}", oo_bins, oo_start, oo_end), "aw", "weight")
h_awph = df.Histo1D(("", ";O_{#alphaW#phi}", oo_bins, oo_start, oo_end), "awph", "weight")
h_abph = df.Histo1D(("", ";O_{#alphaB#phi}", oo_bins, oo_start, oo_end), "abph", "weight")

h_dg1z = df.Histo1D(("", ";O_{#delta g_{1}^{Z}}", oo_bins, oo_start, oo_end), "dg1z", "weight")
h_dkg = df.Histo1D(("", ";O_{#delta#kappa_{#gamma}}", oo_bins, oo_start, oo_end), "dkg", "weight")
h_lz = df.Histo1D(("", ";O_{#lambda_{Z}}", oo_bins, oo_start, oo_end), "lz", "weight")

h_fg1 = df.Histo1D(("", ";O_{f_{1}^{#gamma}}", oo_bins, oo_start, oo_end), "fg1", "weight")
h_fg2 = df.Histo1D(("", ";O_{f_{2}^{#gamma}}", oo_bins, oo_start, oo_end), "fg2", "weight")
h_fg3 = df.Histo1D(("", ";O_{f_{3}^{#gamma}}", oo_bins, oo_start, oo_end), "fg3", "weight")
h_fg4 = df.Histo1D(("", ";O_{f_{4}^{#gamma}}", oo_bins, oo_start, oo_end), "fg4", "weight")
h_fg5 = df.Histo1D(("", ";O_{f_{5}^{#gamma}}", oo_bins, oo_start, oo_end), "fg5", "weight")
h_fg6 = df.Histo1D(("", ";O_{f_{6}^{#gamma}}", oo_bins, oo_start, oo_end), "fg6", "weight")
h_fg7 = df.Histo1D(("", ";O_{f_{7}^{#gamma}}", oo_bins, oo_start, oo_end), "fg7", "weight")

h_fz1 = df.Histo1D(("", ";O_{f_{1}^{Z}}", oo_bins, oo_start, oo_end), "fz1", "weight")
h_fz2 = df.Histo1D(("", ";O_{f_{2}^{Z}}", oo_bins, oo_start, oo_end), "fz2", "weight")
h_fz3 = df.Histo1D(("", ";O_{f_{3}^{Z}}", oo_bins, oo_start, oo_end), "fz3", "weight")
h_fz4 = df.Histo1D(("", ";O_{f_{4}^{Z}}", oo_bins, oo_start, oo_end), "fz4", "weight")
h_fz5 = df.Histo1D(("", ";O_{f_{5}^{Z}}", oo_bins, oo_start, oo_end), "fz5", "weight")
h_fz6 = df.Histo1D(("", ";O_{f_{6}^{Z}}", oo_bins, oo_start, oo_end), "fz6", "weight")
h_fz7 = df.Histo1D(("", ";O_{f_{7}^{Z}}", oo_bins, oo_start, oo_end), "fz7", "weight")

In [19]:
h_enu_m = df.Histo1D("enu_m")
h_pnu_m = df.Histo1D("pnu_m")

In [20]:
h_l_e = df.Histo1D(("", ";E_{e} [GeV]", 300, 0., 150.), "l_MC_lvec_e", "weight")
h_nu_e = df.Histo1D(("", ";E_{#nu} [GeV]", 300, 0., 150.), "enu_MC_lvec_e", "weight")
h_2d_e_e = df.Histo2D(("", ";E_{e-} [GeV];E_{#bar{#nu}} [GeV]", 300, 0., 150., 300, 0., 150.), "e_MC_lvec_e", "enu_MC_lvec_e", "weight")
h_2d_p_e = df.Histo2D(("", ";E_{e+} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "p_MC_lvec_e", "pnu_MC_lvec_e", "weight")
h_2d_lnu_pzpt = df.Histo2D(("", ";P_{z}(e) [GeV];P_{t}(e) [GeV]", 150, -150., 150., 150, 0., 150.), "l_MC_lvec_pz", "l_MC_lvec_pt", "weight")
h_2d_lnu_cosThetapt = df.Histo2D(("", ";cos #theta(e);P_{t}(e) [GeV]", 150, -1., 1., 150, 0., 150.), "l_MC_lvec_cosTheta", "l_MC_lvec_pt", "weight")

h_2d_ep_px = df.Histo2D(("", ";P_{x}(e-) [GeV];P_{x}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_px", "p_MC_lvec_px", "weight")
h_2d_ep_py = df.Histo2D(("", ";P_{y}(e-) [GeV];P_{y}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_py", "p_MC_lvec_py", "weight")
h_2d_ep_pz = df.Histo2D(("", ";P_{z}(e-) [GeV];P_{z}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_pz", "p_MC_lvec_pz", "weight")

h_2d_WW_px = df.Histo2D(("", ";P_{x}(W-) [GeV];P_{x}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_px", "Wp_MC_lvec_px", "weight")
h_2d_WW_py = df.Histo2D(("", ";P_{y}(W-) [GeV];P_{y}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_py", "Wp_MC_lvec_py", "weight")
h_2d_WW_pz = df.Histo2D(("", ";P_{z}(W-) [GeV];P_{z}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_pz", "Wp_MC_lvec_pz", "weight")

h_e_px = df.Histo1D(("", ";P_{x}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_px", "weight")
h_e_py = df.Histo1D(("", ";P_{y}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_py", "weight")
h_e_pz = df.Histo1D(("", ";P_{z}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_pz", "weight")

h_p_px = df.Histo1D(("", ";P_{x}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_px", "weight")
h_p_py = df.Histo1D(("", ";P_{y}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_py", "weight")
h_p_pz = df.Histo1D(("", ";P_{z}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_pz", "weight")

h_CosTheta_W_minus = df.Histo1D(("", ";cos #theta_{W^{-}}", 40, -1., 1.), "CosTheta_W_minus", "weight")
h_CosTheta_W_plus = df.Histo1D(("", ";cos #theta_{W^{+}}", 40, -1., 1.), "CosTheta_W_plus", "weight")
# h_l_theta_prime = df.Histo1D("l_theta_prime", "weight")
h_l_plus_theta_prime = df.Histo1D("p_theta_prime", "weight")
h_l_plus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{+}}", 40, -1., 1.), "p_cosTheta_prime", "weight")
h_l_plus_phi_prime = df.Histo1D(("", ";#phi*_{e+}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "p_phi_prime", "weight")
h_l_minus_theta_prime = df.Histo1D("e_theta_prime", "weight")
h_l_minus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{-}}", 40, -1., 1.), "e_cosTheta_prime", "weight")
h_l_minus_phi_prime = df.Histo1D(("", ";#phi*_{e-}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "e_phi_prime", "weight")

h_l_cosTheta_prime = h_l_plus_cosTheta_prime.Clone()
h_l_cosTheta_prime.Add(h_l_minus_cosTheta_prime.GetPtr())
h_l_cosTheta_prime.SetTitle(";cos #theta*_{e^{#pm}}")
h_l_phi_prime = h_l_plus_phi_prime.Clone()
h_l_phi_prime.Add(h_l_minus_phi_prime.GetPtr())
h_l_phi_prime.SetTitle(";#phi*_{e#pm}")

In [21]:
h_l_phi_prime.SetMinimum(0)
h_l_plus_phi_prime.SetMinimum(0)
h_l_minus_phi_prime.SetMinimum(0)

In [22]:
c_enu_m = ROOT.TCanvas()
h_enu_m.Draw()
c_enu_m.Draw()

c_pnu_m = ROOT.TCanvas()
h_pnu_m.Draw()
c_pnu_m.Draw()

In [23]:
# c_l_e = ROOT.TCanvas()
# h_l_e.Draw()
# c_l_e.Draw()

# c_nu_e = ROOT.TCanvas()
# h_nu_e.Draw()
# c_nu_e.Draw()

# c_2d_ep_px = ROOT.TCanvas()
# h_2d_ep_px.Draw("colz0")
# c_2d_ep_px.Draw()

# c_e_px = ROOT.TCanvas()
# h_e_px.Draw("hist")
# h_p_px.SetLineColor(ROOT.kRed)
# h_p_px.Draw("same hist")
# c_e_px.Draw()

# c_2d_ep_py = ROOT.TCanvas()
# h_2d_ep_py.Draw("colz0")
# c_2d_ep_py.Draw()

# c_e_py = ROOT.TCanvas()
# h_e_py.Draw("hist")
# h_p_py.SetLineColor(ROOT.kRed)
# h_p_py.Draw("same hist")
# c_e_py.Draw()

# c_2d_ep_pz = ROOT.TCanvas()
# h_2d_ep_pz.Draw("colz0")
# c_2d_ep_pz.Draw()

# c_e_pz = ROOT.TCanvas()
# h_e_pz.Draw("hist")
# h_p_pz.SetLineColor(ROOT.kRed)
# h_p_pz.Draw("same hist")
# c_e_pz.Draw()

# c_2d_WW_px = ROOT.TCanvas()
# h_2d_WW_px.Draw("colz0")
# c_2d_WW_px.Draw()

# c_2d_WW_py = ROOT.TCanvas()
# h_2d_WW_py.Draw("colz0")
# c_2d_WW_py.Draw()

# c_2d_WW_pz = ROOT.TCanvas()
# h_2d_WW_pz.Draw("colz0")
# c_2d_WW_pz.Draw()

# c_2d_e_e = ROOT.TCanvas()
# h_2d_e_e.Draw("colz0")
# c_2d_e_e.Draw()

# c_2d_p_e = ROOT.TCanvas()
# h_2d_p_e.Draw("colz0")
# c_2d_p_e.Draw()

# c_2d_lnu_pzpt = ROOT.TCanvas()
# h_2d_lnu_pzpt.Draw("colz0")
# c_2d_lnu_pzpt.Draw()

# c_2d_lnu_cosThetapt = ROOT.TCanvas()
# h_2d_lnu_cosThetapt.Draw("colz0")
# c_2d_lnu_cosThetapt.Draw()

c_CosTheta_W_minus = ROOT.TCanvas()
h_CosTheta_W_minus.Draw()
# stats = h_CosTheta_W_minus.FindObject("stats")
# stats.SetX1NDC(0.2)
# stats.SetX2NDC(0.5)
# stats.SetY1NDC(0.5)
# stats.SetY2NDC(0.8)
c_CosTheta_W_minus.Draw()
c_CosTheta_W_minus.SaveAs(f"{plot_path}/cosThetaWm.pdf")

# c_CosTheta_W_plus = ROOT.TCanvas()
# h_CosTheta_W_plus.Draw()
# c_CosTheta_W_plus.Draw()

# c_l_theta_prime = ROOT.TCanvas()
# h_l_theta_prime.Draw()
# c_l_theta_prime.Draw()

c_l_cosTheta_prime = ROOT.TCanvas()
h_l_cosTheta_prime.Draw()
c_l_cosTheta_prime.Draw()
c_l_cosTheta_prime.SaveAs(f"{plot_path}/cosThetaPrime_l.pdf")

c_l_phi_prime = ROOT.TCanvas()
h_l_phi_prime.Draw()
c_l_phi_prime.Draw()
c_l_phi_prime.SaveAs(f"{plot_path}/PhiPrime_l.pdf")

# c_l_plus_theta_prime = ROOT.TCanvas()
# h_l_plus_theta_prime.Draw()
# c_l_plus_theta_prime.Draw()

c_l_plus_cosTheta_prime = ROOT.TCanvas()
h_l_plus_cosTheta_prime.Draw()
c_l_plus_cosTheta_prime.Draw()

c_l_plus_phi_prime = ROOT.TCanvas()
h_l_plus_phi_prime.Draw()
c_l_plus_phi_prime.Draw()

# c_l_minus_theta_prime = ROOT.TCanvas()
# h_l_minus_theta_prime.Draw()
# c_l_minus_theta_prime.Draw()

c_l_minus_cosTheta_prime = ROOT.TCanvas()
h_l_minus_cosTheta_prime.Draw()
c_l_minus_cosTheta_prime.Draw()

c_l_minus_phi_prime = ROOT.TCanvas()
h_l_minus_phi_prime.SetMinimum(0)
h_l_minus_phi_prime.Draw()
c_l_minus_phi_prime.Draw()

# c_CosTheta_W_minus_LT = ROOT.TCanvas()
# h_CosTheta_W_minus_LT.Draw()
# c_CosTheta_W_minus_LT.Draw()

ERROR in TPDF::Open: Cannot open file:/dev/null/cosThetaWm.pdf
ERROR in TPDF::Open: Cannot open file:/dev/null/cosThetaPrime_l.pdf
ERROR in TPDF::Open: Cannot open file:/dev/null/PhiPrime_l.pdf


In [24]:
c_aw = ROOT.TCanvas()
h_aw.Draw()
c_aw.Draw()
c_aw.SaveAs(f"{oo_plot_name}(")

c_awph = ROOT.TCanvas()
h_awph.Draw()
c_awph.Draw()
c_awph.SaveAs(oo_plot_name)

c_abph = ROOT.TCanvas()
h_abph.Draw()
c_abph.Draw()
c_abph.SaveAs(oo_plot_name)

c_dg1z = ROOT.TCanvas()
h_dg1z.Draw()
c_dg1z.Draw()
c_dg1z.SaveAs(oo_plot_name)

c_dkg = ROOT.TCanvas()
h_dkg.Draw()
c_dkg.Draw()
c_dkg.SaveAs(oo_plot_name)

c_lz = ROOT.TCanvas()
h_lz.Draw()
c_lz.Draw()
c_lz.SaveAs(oo_plot_name)


Info in <TCanvas::Print>: pdf file OO_wz.pdf has been created using the current canvas
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf


In [25]:

c_fg1 = ROOT.TCanvas()
h_fg1.Draw()
c_fg1.Draw()
c_fg1.SaveAs(oo_plot_name)

c_fg2 = ROOT.TCanvas()
h_fg2.Draw()
c_fg2.Draw()
c_fg2.SaveAs(oo_plot_name)

c_fg3 = ROOT.TCanvas()
h_fg3.Draw()
c_fg3.Draw()
c_fg3.SaveAs(oo_plot_name)

c_fg4 = ROOT.TCanvas()
h_fg4.Draw()
c_fg4.Draw()
c_fg4.SaveAs(oo_plot_name)

c_fg5 = ROOT.TCanvas()
h_fg5.Draw()
c_fg5.Draw()
c_fg5.SaveAs(oo_plot_name)

c_fg6 = ROOT.TCanvas()
h_fg6.Draw()
c_fg6.Draw()
c_fg6.SaveAs(oo_plot_name)

c_fg7 = ROOT.TCanvas()
h_fg7.Draw()
c_fg7.Draw()
c_fg7.SaveAs(oo_plot_name)

c_fz1 = ROOT.TCanvas()
h_fz1.Draw()
c_fz1.Draw()
c_fz1.SaveAs(oo_plot_name)

c_fz2 = ROOT.TCanvas()
h_fz2.Draw()
c_fz2.Draw()
c_fz2.SaveAs(oo_plot_name)

c_fz3 = ROOT.TCanvas()
h_fz3.Draw()
c_fz3.Draw()
c_fz3.SaveAs(oo_plot_name)

c_fz4 = ROOT.TCanvas()
h_fz4.Draw()
c_fz4.Draw()
c_fz4.SaveAs(oo_plot_name)

c_fz5 = ROOT.TCanvas()
h_fz5.Draw()
c_fz5.Draw()
c_fz5.SaveAs(oo_plot_name)

c_fz6 = ROOT.TCanvas()
h_fz6.Draw()
c_fz6.Draw()
c_fz6.SaveAs(oo_plot_name)

c_fz7 = ROOT.TCanvas()
h_fz7.Draw()
c_fz7.Draw()
c_fz7.SaveAs(f"{oo_plot_name})")

Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf and file closed
